# Assignment 2

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Linear Regression

Let's set up our workspace and use the **Auto MPG dataset**. This dataset contains several features (such as horsepower, weight, displacement, and acceleration) and a target variable indicating the car's **miles per gallon (MPG)**.

Here, we will model **MPG (continuous outcome)** based on the car's physical and performance characteristics.

In [63]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import seaborn as sns

# Load the Auto MPG dataset
mpg_data = sns.load_dataset('mpg')

# Drop rows with missing values
mpg_data = mpg_data.dropna(subset=['mpg', 'horsepower', 'weight', 'displacement'])

# Display the DataFrame
mpg_data.head()

#### **Question 1:**  
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Auto MPG dataset**:

_(i)_ How many observations (rows) does the dataset contain and also how many variables (columns) does the dataset contain?

In [2]:
# Your answer here...
import seaborn as sns
mpg_data = sns.load_dataset('mpg')
mpg_data = mpg_data.dropna(subset=['mpg', 'horsepower', 'weight', 'displacement'])
mpg_data.head()

Matplotlib is building the font cache; this may take a moment.


mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
3          70    usa              amc rebel sst  
4          70    usa                ford torino

_(ii)_ What is the 'variable type' of the response variable `mpg` (e.g., 'integer', 'category', etc.) and explain what it is in short? (1-2 sentences at max)

In [3]:
# Your answer here...
mpg_data['mpg'].dtype

dtype('float64')

Explanation: The mpg variable is likely a float (decimal number), representing a continuous outcome variable, as it measures miles per gallon, which is a numeric quantity that can have decimal points. 


_(iii)_ Find the 5 rows with the greatest `horsepower`.

In [5]:
# Your answer here...
# mpg_data.nlargest(5, 'horsepower') 

_(iv)_ How many predictor variables do we have (Hint: all variables other than `mpg`)?

In [6]:
# Your answer here...
len(mpg_data.columns) - 1

8

You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Data-visualization

Before we fit and review model outputs, we should visualize our data. Review the code and plot, shown below.

In [ ]:
# Exclude the 'mpg' (target variable) and non-numeric columns from the feature names
feature_names = mpg_data.select_dtypes(include=[float, int]).columns.difference(['mpg'])

# Loop through each numeric feature (column) in mpg_data
for feature in feature_names:
    # Extract the feature column and target variable (mpg)
    X_feature = mpg_data[[feature]].values  # Extract as a 2D array
    y = mpg_data['mpg'].values  # Target variable (mpg)
    
    # Create a scatter plot for the feature against the target (mpg)
    plt.figure(figsize=(6, 4))
    plt.scatter(X_feature, y, label='Data', color='blue')

    # Fit a linear regression model
    lm = LinearRegression()
    lm.fit(X_feature, y)

    # Plot the regression line
    plt.plot(X_feature, lm.predict(X_feature), color='red', label='Regression Line')

    # Add labels and title
    plt.xlabel(feature)
    plt.ylabel('Miles Per Gallon')
    plt.title(f'{feature} vs Miles Per Gallon')

    # Add a legend
    plt.legend()

    # Show the plot
    plt.show()

Answer the following questions:

_(i)_ Describe the associations being plotted ? (i.e., positive association, negative association, no association)

> The associations being plotted in the code represent the relationship between each numerical feature (such as horsepower, weight, displacement, etc.) and the target variable mpg (miles per gallon).

For each feature, a scatter plot is generated, showing individual data points representing cars. The x-axis represents the feature (e.g., horsepower), and the y-axis represents mpg.
A linear regression line is fitted on the scatter plot to show the trend or relationship between the feature and mpg.
In terms of the specific associations:

Positive Association: If the slope of the regression line is upward, it indicates a positive association, meaning as the feature increases, mpg tends to increase as well.
Negative Association: If the slope of the regression line is downward, it indicates a negative association, meaning as the feature increases, mpg tends to decrease.
No Association: If the data points are scattered randomly with no clear trend in the regression line, there may be little to no association between the feature and mpg.
Each feature will have a different type of association depending on how it correlates with mpg. For example, weight may show a negative association, as heavier cars tend to have lower fuel efficiency.

_(ii)_ What concept ‘defines’ the plotted line?

The plotted line is defined by the linear regression model. Specifically, it represents the line of best fit, which is the line that minimizes the sum of the squared differences (errors) between the observed data points and the predicted values (regression line). This line reflects the linear relationship between the feature and the target variable (mpg).

_(iii)_ Do all data points in the dataset fall perfectly along the plotted line? If not, why might there be deviations between the data points and the line, and what do these deviations indicate about the relationship between the variables?

No, not all data points will fall perfectly along the plotted line. The deviations between the data points and the regression line are referred to as residuals or errors. These deviations indicate that there are factors affecting mpg that are not fully explained by the feature being plotted.

If the data points deviate significantly from the line, it suggests that the relationship between the feature and mpg may not be purely linear, or that other factors may influence mpg as well.
The magnitude of these deviations (residuals) can indicate how well the model is fitting the data: larger residuals suggest a poorer fit.

#### **Question 3:**  
#### Model fit  
Now, let’s fit a multivariable linear regression model using the general syntax `lm()`. As above, use **mpg** as the response variable **Y**, and all other variables as the predictors.

**Step 1: Split the dataset into train and test sets, using a 75-25 split. (use random_state=42)**

In [7]:
from sklearn.model_selection import train_test_split

# Define the predictor variables (all numeric columns except 'mpg')
numeric_predictors = mpg_data.select_dtypes(include=[float, int]).drop(columns=['mpg'])

# Define the target variable
y = mpg_data['mpg']

# Split the data (75% training, 25% testing, random state for reproducibility)
X_train, X_test, y_train, y_test = train_test_split(numeric_predictors, y, test_size=0.25, random_state=42)

**Step 2: Fit the linear regression model.**

In [8]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

lm.fit(X_train, y_train)

coefficients_df = pd.DataFrame({
    "predictor": numeric_predictors.columns,
    "slope": lm.coef_,
    "intercept": [lm.intercept_] * len(lm.coef_)
})

# Display the coefficients DataFrame
print(coefficients_df)

numeric_predictors = 🤷‍♂️


# Create a DataFrame containing the slope (coefficients) and intercept
coefficients_df = pd.DataFrame({
    "predictor": numeric_predictors.columns,
    "slope": lm.coef_,
    "intercept": [lm.intercept_] * len(lm.coef_)
})

# Display the coefficients DataFrame
print(coefficients_df)

# lm.coef_ gives the coefficients for each predictor (change in miles per gallon per unit change in each predictor variable)
# lm.intercept_ gives the intercept b_0 (the predicted miles per gallon when all predictors are set to 0)

SyntaxError: invalid character '🤷' (U+1F937) (327919923.py, line 16)

#### **Question 4:** 
#### RMSPE
**Step 3. Finally, we predict on the test data set to assess how well our model does.** 

We will evaluate our final model's test error measured by RMSPE.

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

numeric_predictors = mpg_data.select_dtypes(include=[float, int]).drop(columns=['mpg'])

y = mpg_data['mpg']

X_train, X_test, y_train, y_test = train_test_split(numeric_predictors, y, test_size=0.25, random_state=42)

lm = LinearRegression()

lm.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
import numpy as np

y_pred = lm.predict(X_test)

percentage_error = (y_test - y_pred) / y_test

rmspe = np.sqrt(np.mean(percentage_error ** 2))

print("RMSPE:", rmspe)

RMSPE: 0.1605046358728446


# Criteria

| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for the number of variables, observations, and data types. | Data inspection is missing or incomplete.         |
| **Data Visualization**                                 | Visualizations (e.g., scatter plots, histograms) are properly interepreted to explore the relationships between variables. | Data visualization were not correctly interpreted. |
| **Model Initialization**                               | The linear regression model is correctly initialized.            | The linear regression model is not initialized or is incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using appropriate metrics (e.g., RMSE). | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
